### Mathematical Optimization: First Exercise with Xpress and SCIP

Exercises will be done in Jupyter notebooks. In this tutorial, you will

1. model an introductory linear program using Xpress Python
2. model the same program with pySCIPopt

#### Problem formulation

Try to find the optimal solution to the following problem:

$$
\begin{align*}
    \min x + y \\
    x + 2y \geq 5\\
    4x + y \geq 6
\end{align*}
$$
You might remember this problem from the registration process. Do you also remember the solution?

#### Xpress
Let's start with the Xpress implementation


In [2]:
# import the Xpress Python module.
import xpress as xp

Code cells like the one above are editable, markdown cells like this are not. You need to execute a cell in order for the code in it to take action. One way of doing so is to click on the cell, and then select Cells --> Run Cells from the menu. Do this now for the cell above.
 
Let's now start with your first line of code. The code below creates a first variable x. Use the same function to create a variable y in the code cell below.



In [13]:
# create two variables x and y
x = xp.var('x', lb = -xp.infinity)
y = xp.var('y', lb = -xp.infinity)
#...TODO...

print(f"1st variable: name {x.name} lb {x.lb} ub {x.ub} ")
print(f"2nd variable: name {y.name} lb {y.lb} ub {y.ub} ")

1st variable: name x lb -1e+20 ub 1e+20 
2nd variable: name y lb -1e+20 ub 1e+20 


Let's take a big leap and create the whole problem. Since it is tiny, we will use a compact declaration. Checkout the documentation of the Xpress Python interface:
     https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/chModeling.html 

Go to 'Creating a problem' and look for the example at the bottom of the page. Try to adapt it to our model.



In [14]:
# create a problem using the one-liner notation that first 
# adds the variables, then the constraints, and finally, the objective function
p = xp.problem(x, y, x+2*y >= 5, 4*x+y >= 6, x+y, name = 'registration_problem')

p.write("register","lp")
with open("register.lp", "r") as file_:
    print("".join(file_.readlines()))


\Problem name: registration_problem                                             
\FICO Xpress v8.9.0, Hyper, written 8:16:13, Sep 14, 2020

Minimize
 x + y

Subject To
R15: x + 2 y >= 5 
R16: 4 x + y >= 6 

Bounds
x free
y free

End



Search the documentation on how you solve a model (or take an educated guess).



In [15]:
# solve it
#...TODO...
p.solve()

FICO Xpress v8.9.0, Hyper, solve started 8:16:20, Sep 14, 2020
Heap usage: 331KB (peak 331KB, 883KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Minimizing LP registration_problem
Original problem has:
         2 rows            2 cols            4 elements
Presolved problem has:
         0 rows            0 cols            0 elements
Presolve finished in 0 seconds
Heap usage: 331KB (peak 343KB, 885KB system)
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0          3.000000      D      0     0        .000000     0
Uncrunching matrix
Optimal solution found
Dual solved problem
  0 simplex iterations in 0s

Final objective                       : 3.000000000000000e+00
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0


Now store the optimal objective and solution vector into two local variables obj and sol.



In [34]:
# check solution
#...TODO...
sol = p.getSolution()
obj = p.getObjVal()

print("solution {} with objective value {}.".format(sol, obj))

solution [1.0, 2.0] with objective value 3.0.


Congratulations, you solved your first model with the Xpress Python interface.

The one-line declaration of a problem is handy, but there is a more general way: we can create constraints and objective functions, then create an empty problem and add each object independently with addVariable, addConstraint, and setObjective.


In [18]:
x = xp.var('x', lb = -xp.infinity)
y = xp.var('y', lb = -xp.infinity)

con1 = x+2*y >= 5
con2 = 4*x+y >= 6
objective = x+y

m = xp.problem()

# Add variables, then add constraints and set the objective
m.addVariable(x)
m.addVariable(y)
m.addConstraint(con1)
m.addConstraint(con2)
m.setObjective(objective, sense = xp.minimize)
# ...TODO...

Finally, solve the problem and display the solution (see above).

In [19]:
# Solve
# ...TODO...
# Print solution (should be the same as the first solve)
m.solve()

FICO Xpress v8.9.0, Hyper, solve started 8:21:25, Sep 14, 2020
Heap usage: 331KB (peak 331KB, 1208KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Minimizing LP noname
Original problem has:
         2 rows            2 cols            4 elements
Presolved problem has:
         0 rows            0 cols            0 elements
Presolve finished in 0 seconds
Heap usage: 331KB (peak 343KB, 1210KB system)
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0          3.000000      D      0     0        .000000     0
Uncrunching matrix
Optimal solution found
Dual solved problem
  0 simplex iterations in 0s

Final objective                       : 3.000000000000000e+00
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0


#### SCIP

Now that you know how to model and solve problems with Xpress, let us do the same with **SCIP** now


In [20]:
from pyscipopt import Model

# initialize model
model = Model()
# make scip output visible in jupyter notebook
model.redirectOutput()

First, we add the variables. See how the first variable is added and add the second accordingly.

In [21]:
# add variables
x = model.addVar('x', lb = None)
y = model.addVar('y',lb = None)
#...TODO...



Next, we add an objective and the two constraints. Consult the online documentation at http://scip-interfaces.github.io/PySCIPOpt/docs/html/md_README.html on how to do this.

In [22]:
# add objective 
#...TODO...
model.setObjective(x+y)

# add constraints
#...TODO...
model.addCons(x+2*y >= 5)
model.addCons(4*x+y >= 6)
# check problem
model.writeProblem()

STATISTICS
  Problem name     : model
  Variables        : 2 (0 binary, 0 integer, 0 implicit integer, 2 continuous)
  Constraints      : 0 initial, 2 maximal
OBJECTIVE
  Sense            : minimize
VARIABLES
  [continuous] <x>: obj=1, original bounds=[-inf,+inf]
  [continuous] <y>: obj=1, original bounds=[-inf,+inf]
CONSTRAINTS
  [linear] <c1>: <x>[C] +2<y>[C] >= 5;
  [linear] <c2>:  +4<x>[C] +<y>[C] >= 6;
END
wrote problem to file b'model.cip'


Solve the model and print the optimal solution vector and its objective value. For the latter, use the getSolVal function; consult the documentation on its syntax.

In [36]:
# solve problem
#...TODO...
model.optimize()

# check solution
#...TODO...
sol = model.getBestSol()
obj = model.getObjVal()

print("solution {} with objective value {}.".format(sol, obj))

solution {'t_x': 1.0, 't_y': 2.0} with objective value 3.0.
